In [2]:
from InteractionDetection import *
from sklearn.linear_model import LinearRegression
from Metrics import *
from TimewiseKfold import *
import numpy as np

In [3]:
scaled_folds, PROVINCES_NUM = splitting_data(is_scaled=True, region=2)

In [18]:
selected_features = ['Rdays_VII', 'Rdays_X', 'Srad_I', 'Srad_IX', 'Srad_X']

#1st iteration
selected_features += ['Eva_I', 'Rdays_III', 'Tmax_avg_IX', 'Tmin_II']

#2st iteration
selected_features += ['Eva_IV']

#3rd iteration
selected_features += ['Prcp_max_XII']

#4th iteration
selected_features += ['Prcp_X']

coefs = [-2.20414717, -9.79981796, -1.40394979, -1.99214848, -0.0558246 ,
        -0.53806973, -1.29472508,  1.06818365,  3.43466945,  4.57856107,
         1.51546732,  1.18043713,  0.22542275, 19.97290719, -1.15140017,
        -5.17811202,  0.17753217, -0.25629687, -0.69383557]

# for i in range(len(selected_features)):
#    print(selected_features[i], coefs[i])

alphas_res = [
   np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0. , 0.5, 0. , 0.5, 0.5]), 
   np.array([0.5, 0. , 0.5, 0.5, 0.5, 0.5, 0.5, 0. , 0.5, 0.5, 0.5, 0.5]), 
   np.array([0.5, 0. , 0.5, 0. , 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]), 
   np.array([0.5, 0.5, 0.5, 0.5, 0. , 0.5, 0.5, 0.5, 0.5, 1. , 0.5, 0.5]), 
   np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1. , 0. , 0.5, 0.5]), 
   np.array([0. , 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1. , 0.5, 0.5]), 
   np.array([0.5, 1. , 0.5, 0.5, 0.5, 0.5, 0. , 0.5, 0.5, 0.5, 0.5, 0.5]) 
]

for (order, alpha) in enumerate(alphas_res):
   for i in range(0, len(alpha)):
      if (alpha[i] != 0.5):
         print(selected_features[i], end=' ')
   print(coefs[12 + order])
      
   # print()

Tmax_avg_IX Eva_IV 0.22542275
Rdays_X Tmax_avg_IX 19.97290719
Rdays_X Srad_IX -1.15140017
Srad_X Eva_IV -5.17811202
Tmin_II Eva_IV 0.17753217
Rdays_VII Eva_IV -0.25629687
Rdays_X Rdays_III -0.69383557


In [13]:
selected_features = ['Rdays_VII', 'Rdays_X', 'Srad_I', 'Srad_IX', 'Srad_X']

#1st iteration
selected_features += ['Eva_I', 'Rdays_III', 'Tmax_avg_IX', 'Tmin_II']

#2st iteration
selected_features += ['Eva_IV']

#3rd iteration
selected_features += ['Prcp_max_XII']

#4th iteration
selected_features += ['Prcp_X']

alphas_res = [
   np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0. , 0.5, 0. , 0.5, 0.5]), 
   np.array([0.5, 0. , 0.5, 0.5, 0.5, 0.5, 0.5, 0. , 0.5, 0.5, 0.5, 0.5]), 
   np.array([0.5, 0. , 0.5, 0. , 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]), 
   np.array([0.5, 0.5, 0.5, 0.5, 0. , 0.5, 0.5, 0.5, 0.5, 1. , 0.5, 0.5]), 
   np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1. , 0. , 0.5, 0.5]), 
   np.array([0. , 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1. , 0.5, 0.5]), 
   np.array([0.5, 1. , 0.5, 0.5, 0.5, 0.5, 0. , 0.5, 0.5, 0.5, 0.5, 0.5]) 
]

deltas_res = [
   np.array([0., 0., 0., 0., 1., 0.]), 
   np.array([0., 0., 1., 0., 0., 0.]), 
   np.array([0., 0., 0., 1., 0., 0.]), 
   np.array([1., 0., 0., 0., 0., 0.]), 
   np.array([0., 0., 0., 0., 1., 0.]), 
   np.array([0., 0., 0., 0., 1., 0.]), 
   np.array([0., 1., 0., 0., 0., 0.])
]


# heuristic_interaction_detection(scaled_folds=scaled_folds, selected_features=selected_features, K=6, alphas_res=alphas_res, deltas_res=deltas_res, M=2, PROVINCES_NUM=PROVINCES_NUM)
 
transformed_folds = transform_data(scaled_folds=scaled_folds, selected_features=selected_features, alphas=alphas_res, deltas=deltas_res)

rmse_train, rmse_validatioin = RMSE_FOLDS(transformed_folds=transformed_folds, PROVINCES_NUM=PROVINCES_NUM)

print(rmse_train, rmse_validatioin)

coef = []

table = metrics_table()

for (i, fold) in enumerate(transformed_folds):
    X_train, X_test, y_train, y_test = split_train_test(fold=fold)
   #  X_train, X_validation, y_train, y_validation = split_train_validation(X=X_train, y=y_train, PROVINCES_NUM=PROVINCES_NUM)
    
    
    linear_model = LinearRegression(n_jobs=-1).fit(X=X_train, y=y_train)
    
   #  print(np.sqrt(mean_squared_error(y_true=y_test, y_pred=linear_model.predict(X_test))))
    
    row = [MAXIMUM_YEAR - FOLD_NUM + 1 + i]
    
    row += metrics_calculation(y_actual=y_test, y_pred=linear_model.predict(X_test), y_train=y_train)
    
    table.loc[len(table)] = row
    
    if (i == 2):
       coef = linear_model.coef_
    
    del linear_model
    
table
coef

2.093475160828057 1.1998343396018514


array([[-2.20414717, -9.79981796, -1.40394979, -1.99214848, -0.0558246 ,
        -0.53806973, -1.29472508,  1.06818365,  3.43466945,  4.57856107,
         1.51546732,  1.18043713,  0.22542275, 19.97290719, -1.15140017,
        -5.17811202,  0.17753217, -0.25629687, -0.69383557]])